Referred doc-https://huggingface.co/learn/nlp-course/chapter6/5?fw=pt#implementing-bpe

In [1]:
corpus = [
    "This is a Natural Language Processing course.",
    "We will learn about tokenization.",
    "Here, we will explore Byte-Pair Encoding tokenizers",
    "We will be able to understand how tokenizers are trained and tokens are generated.",
]

corpus

['This is a Natural Language Processing course.',
 'We will learn about tokenization.',
 'Here, we will explore Byte-Pair Encoding tokenizers',
 'We will be able to understand how tokenizers are trained and tokens are generated.']

Next, we need to pre-tokenize that corpus into words. Since we are replicating a BPE tokenizer (like GPT-2), we will use the gpt2 tokenizer for the pre-tokenization.

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

Then we compute the frequencies of each word in the corpus as we do the pre-tokenization

In [3]:
from collections import defaultdict

word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)

    words = [word for word, offset in words_with_offsets]

    for word in words:
        word_freqs[word] += 1

word_freqs

defaultdict(int,
            {'This': 1,
             'Ġis': 1,
             'Ġa': 1,
             'ĠNatural': 1,
             'ĠLanguage': 1,
             'ĠProcessing': 1,
             'Ġcourse': 1,
             '.': 3,
             'We': 2,
             'Ġwill': 3,
             'Ġlearn': 1,
             'Ġabout': 1,
             'Ġtokenization': 1,
             'Here': 1,
             ',': 1,
             'Ġwe': 1,
             'Ġexplore': 1,
             'ĠByte': 1,
             '-': 1,
             'Pair': 1,
             'ĠEncoding': 1,
             'Ġtokenizers': 2,
             'Ġbe': 1,
             'Ġable': 1,
             'Ġto': 1,
             'Ġunderstand': 1,
             'Ġhow': 1,
             'Ġare': 2,
             'Ġtrained': 1,
             'Ġand': 1,
             'Ġtokens': 1,
             'Ġgenerated': 1})

The next step is to compute the base vocabulary, formed by all the characters used in the corpus

In [4]:
alphabet = []

for word in word_freqs.keys():

    for letter in word:

        if letter not in alphabet:
            alphabet.append(letter)

alphabet.sort()

print(alphabet)

[',', '-', '.', 'B', 'E', 'H', 'L', 'N', 'P', 'T', 'W', 'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'k', 'l', 'n', 'o', 'p', 'r', 's', 't', 'u', 'w', 'x', 'y', 'z', 'Ġ']


In [5]:
len(alphabet)

33

We also add the special tokens used by the model at the beginning of that vocabulary. In the case of GPT-2, the only special token is "<|endoftext|>"

In [6]:
vocab = ["<|endoftext|>"] + alphabet.copy()

vocab

['<|endoftext|>',
 ',',
 '-',
 '.',
 'B',
 'E',
 'H',
 'L',
 'N',
 'P',
 'T',
 'W',
 'a',
 'b',
 'c',
 'd',
 'e',
 'g',
 'h',
 'i',
 'k',
 'l',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'w',
 'x',
 'y',
 'z',
 'Ġ']

We now need to split each word into individual characters, to be able to start training

In [7]:
splits = {word: [c for c in word] for word in word_freqs.keys()}

splits

{'This': ['T', 'h', 'i', 's'],
 'Ġis': ['Ġ', 'i', 's'],
 'Ġa': ['Ġ', 'a'],
 'ĠNatural': ['Ġ', 'N', 'a', 't', 'u', 'r', 'a', 'l'],
 'ĠLanguage': ['Ġ', 'L', 'a', 'n', 'g', 'u', 'a', 'g', 'e'],
 'ĠProcessing': ['Ġ', 'P', 'r', 'o', 'c', 'e', 's', 's', 'i', 'n', 'g'],
 'Ġcourse': ['Ġ', 'c', 'o', 'u', 'r', 's', 'e'],
 '.': ['.'],
 'We': ['W', 'e'],
 'Ġwill': ['Ġ', 'w', 'i', 'l', 'l'],
 'Ġlearn': ['Ġ', 'l', 'e', 'a', 'r', 'n'],
 'Ġabout': ['Ġ', 'a', 'b', 'o', 'u', 't'],
 'Ġtokenization': ['Ġ',
  't',
  'o',
  'k',
  'e',
  'n',
  'i',
  'z',
  'a',
  't',
  'i',
  'o',
  'n'],
 'Here': ['H', 'e', 'r', 'e'],
 ',': [','],
 'Ġwe': ['Ġ', 'w', 'e'],
 'Ġexplore': ['Ġ', 'e', 'x', 'p', 'l', 'o', 'r', 'e'],
 'ĠByte': ['Ġ', 'B', 'y', 't', 'e'],
 '-': ['-'],
 'Pair': ['P', 'a', 'i', 'r'],
 'ĠEncoding': ['Ġ', 'E', 'n', 'c', 'o', 'd', 'i', 'n', 'g'],
 'Ġtokenizers': ['Ġ', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r', 's'],
 'Ġbe': ['Ġ', 'b', 'e'],
 'Ġable': ['Ġ', 'a', 'b', 'l', 'e'],
 'Ġto': ['Ġ', 't', 'o'

Now that we are ready for training, let’s write a function that computes the frequency of each pair. We’ll need to use this at each step of the training:

In [8]:
def compute_pair_freqs(splits):

    pair_freqs = defaultdict(int)

    for word, freq in word_freqs.items():
        split = splits[word]

        # Single letter word e.g. "a", "I", "."
        if len(split) == 1:
            continue

        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])

            pair_freqs[pair] += freq

    return pair_freqs

Viewing all pairs with corresponding frequencies

In [9]:
pair_freqs = compute_pair_freqs(splits)

pair_freqs

defaultdict(int,
            {('T', 'h'): 1,
             ('h', 'i'): 1,
             ('i', 's'): 2,
             ('Ġ', 'i'): 1,
             ('Ġ', 'a'): 6,
             ('Ġ', 'N'): 1,
             ('N', 'a'): 1,
             ('a', 't'): 3,
             ('t', 'u'): 1,
             ('u', 'r'): 2,
             ('r', 'a'): 3,
             ('a', 'l'): 1,
             ('Ġ', 'L'): 1,
             ('L', 'a'): 1,
             ('a', 'n'): 3,
             ('n', 'g'): 3,
             ('g', 'u'): 1,
             ('u', 'a'): 1,
             ('a', 'g'): 1,
             ('g', 'e'): 2,
             ('Ġ', 'P'): 1,
             ('P', 'r'): 1,
             ('r', 'o'): 1,
             ('o', 'c'): 1,
             ('c', 'e'): 1,
             ('e', 's'): 1,
             ('s', 's'): 1,
             ('s', 'i'): 1,
             ('i', 'n'): 3,
             ('Ġ', 'c'): 1,
             ('c', 'o'): 2,
             ('o', 'u'): 2,
             ('r', 's'): 4,
             ('s', 'e'): 1,
             ('W', 'e'): 2,
   

Finding pair with max frequency

In [10]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('Ġ', 'a') 6


Adding the pair to vocabulary

In [11]:
merges = {("Ġ", "a"): "Ġa"}

vocab.append("Ġa")

vocab

['<|endoftext|>',
 ',',
 '-',
 '.',
 'B',
 'E',
 'H',
 'L',
 'N',
 'P',
 'T',
 'W',
 'a',
 'b',
 'c',
 'd',
 'e',
 'g',
 'h',
 'i',
 'k',
 'l',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'w',
 'x',
 'y',
 'z',
 'Ġ',
 'Ġa']

To continue, we need to apply that merge in our splits dictionary. Defining a function for merging pairs

In [12]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]

        # Single letter word e.g. "a", "I", "."
        if len(split) == 1:
            continue

        i = 0

        while i < len(split) - 1:

            if split[i] == a and split[i + 1] == b:
                # Merge the terms
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1

        splits[word] = split

    return splits

And we can have a look at the result of the first merge

In [13]:
splits = merge_pair("Ġ", "a", splits)

print(splits["Ġabout"])

['Ġa', 'b', 'o', 'u', 't']


In [14]:
splits = merge_pair("e", "n", splits)

print(splits["Ġtokenizers"])

['Ġ', 't', 'o', 'k', 'en', 'i', 'z', 'e', 'r', 's']


In [15]:
splits = merge_pair("e", "r", splits)

print(splits["Ġtokenizers"])

['Ġ', 't', 'o', 'k', 'en', 'i', 'z', 'er', 's']


Now we have everything we need to loop until we have learned all the merges we want. Taking vocabulary size of 120.

In [16]:
vocab_size = 120

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)

    best_pair, max_freq = "", None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq

    splits = merge_pair(*best_pair, splits)

    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

As a result, we’ve learned 16 merge rules (the initial vocabulary had a size of 33 characters in the alphabet, plus the special token):

In [17]:
merges

{('Ġ', 'a'): 'Ġa',
 ('Ġ', 't'): 'Ġt',
 ('Ġt', 'o'): 'Ġto',
 ('Ġ', 'w'): 'Ġw',
 ('Ġto', 'k'): 'Ġtok',
 ('Ġtok', 'en'): 'Ġtoken',
 ('a', 't'): 'at',
 ('n', 'g'): 'ng',
 ('Ġw', 'i'): 'Ġwi',
 ('Ġwi', 'l'): 'Ġwil',
 ('Ġwil', 'l'): 'Ġwill',
 ('Ġtoken', 'i'): 'Ġtokeni',
 ('Ġtokeni', 'z'): 'Ġtokeniz',
 ('r', 'e'): 're',
 ('er', 's'): 'ers',
 ('n', 'd'): 'nd',
 ('i', 's'): 'is',
 ('u', 'r'): 'ur',
 ('i', 'ng'): 'ing',
 ('c', 'o'): 'co',
 ('W', 'e'): 'We',
 ('l', 'e'): 'le',
 ('Ġa', 'b'): 'Ġab',
 ('a', 'i'): 'ai',
 ('Ġtokeniz', 'ers'): 'Ġtokenizers',
 ('Ġa', 're'): 'Ġare',
 ('e', 'd'): 'ed',
 ('T', 'h'): 'Th',
 ('Th', 'is'): 'This',
 ('Ġ', 'is'): 'Ġis',
 ('Ġ', 'N'): 'ĠN',
 ('ĠN', 'at'): 'ĠNat',
 ('ĠNat', 'ur'): 'ĠNatur',
 ('ĠNatur', 'a'): 'ĠNatura',
 ('ĠNatura', 'l'): 'ĠNatural',
 ('Ġ', 'L'): 'ĠL',
 ('ĠL', 'a'): 'ĠLa',
 ('ĠLa', 'ng'): 'ĠLang',
 ('ĠLang', 'u'): 'ĠLangu',
 ('ĠLangu', 'a'): 'ĠLangua',
 ('ĠLangua', 'g'): 'ĠLanguag',
 ('ĠLanguag', 'e'): 'ĠLanguage',
 ('Ġ', 'P'): 'ĠP',
 ('ĠP', 'r'): '

And the vocabulary is composed of the special token, the initial alphabet, and all the results of the merges

In [18]:
vocab

['<|endoftext|>',
 ',',
 '-',
 '.',
 'B',
 'E',
 'H',
 'L',
 'N',
 'P',
 'T',
 'W',
 'a',
 'b',
 'c',
 'd',
 'e',
 'g',
 'h',
 'i',
 'k',
 'l',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'w',
 'x',
 'y',
 'z',
 'Ġ',
 'Ġa',
 'Ġt',
 'Ġto',
 'Ġw',
 'Ġtok',
 'Ġtoken',
 'at',
 'ng',
 'Ġwi',
 'Ġwil',
 'Ġwill',
 'Ġtokeni',
 'Ġtokeniz',
 're',
 'ers',
 'nd',
 'is',
 'ur',
 'ing',
 'co',
 'We',
 'le',
 'Ġab',
 'ai',
 'Ġtokenizers',
 'Ġare',
 'ed',
 'Th',
 'This',
 'Ġis',
 'ĠN',
 'ĠNat',
 'ĠNatur',
 'ĠNatura',
 'ĠNatural',
 'ĠL',
 'ĠLa',
 'ĠLang',
 'ĠLangu',
 'ĠLangua',
 'ĠLanguag',
 'ĠLanguage',
 'ĠP',
 'ĠPr',
 'ĠPro',
 'ĠProc',
 'ĠProce',
 'ĠProces',
 'ĠProcess',
 'ĠProcessing',
 'Ġco',
 'Ġcour',
 'Ġcours',
 'Ġcourse',
 'Ġle',
 'Ġlea',
 'Ġlear',
 'Ġlearn',
 'Ġabo',
 'Ġabou',
 'Ġabout',
 'Ġtokenizat',
 'Ġtokenizati',
 'Ġtokenizatio',
 'Ġtokenization',
 'Her',
 'Here',
 'Ġwe',
 'Ġe',
 'Ġex',
 'Ġexp',
 'Ġexpl',
 'Ġexplo',
 'Ġexplore',
 'ĠB',
 'ĠBy',
 'ĠByt',
 'ĠByte',
 'Pai',
 'Pair',
 'ĠE',
 'ĠE

To tokenize a new text, we pre-tokenize it, split it, then apply all the merge rules learned

In [23]:
def tokenize(text):

    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]

    splits = [[l for l in word] for word in pre_tokenized_text]

    # For every pair merged adjust splits for the tokens
    for pair, merge in merges.items():
        # Iterate over the splits for each token
        for idx, split in enumerate(splits):

            i = 0
            while i < len(split) - 1:
                # If splits match the pair that was merged, add the merge
                # to the split
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1

            splits[idx] = split

    return sum(splits, [])

In [24]:
tokenize("This is an implementation of the Byte-pair Encoding tokenizer.")

['This',
 'Ġis',
 'Ġa',
 'n',
 'Ġ',
 'i',
 'm',
 'p',
 'le',
 'm',
 'e',
 'n',
 't',
 'at',
 'i',
 'o',
 'n',
 'Ġ',
 'o',
 'f',
 'Ġt',
 'h',
 'e',
 'ĠByte',
 '-',
 'p',
 'ai',
 'r',
 'ĠEncoding',
 'Ġtok',
 'e',
 'n',
 'i',
 'z',
 'e',
 'r',
 '.']